In [1]:
from models.composer.generate import Generator
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import array_ops

configs = {
    'model_path': '/home/richhiey/Desktop/workspace/projects/virtual_musicians/cache/test_model2',
    'vocab_size': 389,
    'emb_size': 128,
    'lstm_units': 512,
    'dense_units': 64,
    'max_timesteps': 256
}

g = Generator(configs)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(8, 256)]                0         
_________________________________________________________________
embedding (Embedding)        (8, 256, 128)             49792     
_________________________________________________________________
lstm (LSTM)                  (8, 256, 512)             1312768   
_________________________________________________________________
lstm_1 (LSTM)                (8, 256, 512)             2099200   
_________________________________________________________________
dense (Dense)                (8, 256, 64)              32832     
_________________________________________________________________
dense_1 (Dense)              (8, 256, 389)             25285     
Total params: 3,519,877
Trainable params: 3,519,877
Non-trainable params: 0
____________________________________________

In [2]:
DATA_PATH = '/home/richhiey/Desktop/workspace/projects/virtual_musicians/data-store/POP909-Dataset/data_process/pop909-event-token.npy'
data = np.load(DATA_PATH, allow_pickle=True)
print(len(data))
melodies = [d['MELODY'] for d in data]

def generator_fn():
    for melody in melodies:
        yield tf.convert_to_tensor([melody])

dataset = tf.data.Dataset.from_generator(generator_fn, tf.int64, tf.TensorShape([1, None]))

for data in dataset:
    x = data
    print(x)
    print(x.numpy())
    break

boundaries = [128 * (i+1) for i in range(20)]
batch_sizes = [8] * (len(boundaries) + 1)

def _element_length_fn(x, y=None):
    return array_ops.shape(x)[1]

dataset = dataset.apply(
    tf.data.experimental.bucket_by_sequence_length(
        _element_length_fn,
        boundaries,
        batch_sizes,
        drop_remainder=True,
        pad_to_bucket_boundary=False
    )
)

print(dataset)
for x in dataset:
    print(x)
    break
    

g.train(dataset, {'num_epochs':1})

909
tf.Tensor([[355 355 355 ...  73 291 201]], shape=(1, 2221), dtype=int64)
[[355 355 355 ...  73 291 201]]
<_GroupByWindowDataset shapes: (None, 1, None), types: tf.int64>
tf.Tensor(
[[[355 355 355 ...   0   0   0]]

 [[355 355 355 ...   0   0   0]]

 [[355 355 355 ...   0   0   0]]

 ...

 [[355 355 355 ...   0   0   0]]

 [[355 355 355 ...   0   0   0]]

 [[355 355 355 ...   0   0   0]]], shape=(8, 1, 1514), dtype=int64)
tf.Tensor([   8    1 1514], shape=(3,), dtype=int32)


InvalidArgumentError: slice index 0 of dimension 0 out of bounds. [Op:StridedSlice] name: functional_1/lstm/strided_slice/